In [6]:
import gymnasium as gym
import random
from IPython.display import clear_output
%config NotebookApp.iopub_msg_rate_limit=10000
import time

In [7]:
desc = ["SFFF", "FHHH", "FFFF", "HFHF", "FFGF"]
mazeSize = [len(desc),len(desc[0])]

statePositions = [[] for _ in range(mazeSize[0])]
stateNum = 0
for i in range(mazeSize[0]):
    for j in range(mazeSize[1]):
        statePositions[i].append(stateNum)
        stateNum += 1
        
giftState = -1
gift_found = False
for i in range(len(desc)):
    if gift_found:
        break
    for j in range(len(desc[i])):
        giftState += 1
        if desc[i][j] == 'G':
            gift_found = True
            break
            
print(giftState)
print(statePositions)

env = gym.make('FrozenLake-v1', desc=desc, map_name="5x5", is_slippery=False, render_mode="human") 

18
[[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11], [12, 13, 14, 15], [16, 17, 18, 19]]


In [8]:
def resetTable():
    global qTable_1
    qTable_1 = {}
    for i in range(mazeSize[0]*mazeSize[1]):
        qTable_1[i] = [0,0,0,0] 
    global currentState
    currentState = 0
    
def get_Position(state):
    for i in range(len(statePositions)):
        for j in range(len(statePositions[i])):
            if statePositions[i][j] == state:
                return i, j
            
def calcReward(state, nextState):
    y1, x1 = get_Position(state)
    y2, x2 = get_Position(nextState)
    y3, x3 = get_Position(giftState)
    
    currentDist = (((y3 - y1)**2)+((x3 - x1)**2))**0.5
    nextDist = (((y3 - y2)**2)+((x3 - x2)**2))**0.5
    
    changeInDist = currentDist-nextDist
    return changeInDist

def calcPossibleMoves(state):
    global qTable_1
    possibleMoves = []
    
    if state == 0:
        return [1,2]
    
    if (state+1) % mazeSize[1] != 0:
        possibleMoves.append(2)
        
    if (state+1) % mazeSize[1] != 1:
        possibleMoves.append(0)
        
    if state > (mazeSize[1]-1):
        possibleMoves.append(3)
    
    if state < ((mazeSize[0] * mazeSize[1]) - mazeSize[1]):
        possibleMoves.append(1)
        
    return possibleMoves

def nextStep(state):
    global qTable_1
    possMoves = calcPossibleMoves(state)
    
    if random.random() < epsilonValue:
        nextMove = random.choice(possMoves)
    else:
        qValues = {}
        for move in possMoves:
            qValues[move] = qTable_1[state][move]

        maxValue = max(qValues.values())
        minValue = min(qValues.values())
        count_max = sum(1 for value in qValues.values() if value == maxValue)
        count_min = sum(1 for value in qValues.values() if value == minValue)

        if count_max > 1 and count_max < len(possMoves):
            nextMove = random.choice([move for move in possMoves if qValues[move] != minValue])
        elif count_max == len(possMoves):
            nextMove = random.choice(possMoves)
        else:
            nextMove = max(qValues, key=qValues.get)
    return nextMove

def pathFound():
    currentState = 0
    for i in range(mazeSize[0]*mazeSize[1]-1):
        bestDirection = None
        if max(qTable_1[currentState]) > 0:
            bestDirection = qTable_1[currentState].index(max(qTable_1[currentState]))
        newState = 0
        if bestDirection == 0 and 0 in calcPossibleMoves(currentState):
            newState = currentState - 1
        elif bestDirection == 1 and 1 in calcPossibleMoves(currentState):
            newState = currentState + mazeSize[1]
        elif bestDirection == 2 and 2 in calcPossibleMoves(currentState):
            newState = currentState + 1
        elif bestDirection == 3 and 3 in calcPossibleMoves(currentState):
            newState = currentState - mazeSize[1]

        if newState == giftState:
            return True
        currentState = newState
    return False

def updateTable_qLearning(direction, nextState, reward):
    global qTable_1
    global currentState
    didConverge = False
    updated = qTable_1[currentState][direction] + alpha*(reward + (gamma*max(qTable_1[nextState])) - qTable_1[currentState][direction])
    changeInQ = round(abs(qTable_1[currentState][direction] - updated),5)
    if changeInQ < convergenceThresh:
        if changeInQ > 0 and pathFound():
            didConverge = True
    qTable_1[currentState][direction] = updated
    timesVisited[currentState] += 1
    currentState = nextState
    return didConverge, changeInQ

def updateTable_sarsa(direction, nextState, reward):
    global qTable_1
    global currentState
    didConverge = False
    nextDirection = nextStep(nextState)
    updated = qTable_1[currentState][direction] + alpha*(reward + (gamma*qTable_1[nextState][nextDirection]) - qTable_1[currentState][direction])
    changeInQ = round(abs(qTable_1[currentState][direction] - updated),5)
    if changeInQ < convergenceThresh:
        if changeInQ > 0 and pathFound():
            didConverge = True
    qTable_1[currentState][direction] = updated
    timesVisited[currentState] += 1
    currentState = nextState
    return didConverge, changeInQ, nextDirection

def updateTable_monteCarlo(steps, totalReward):
    global qTable_1
    global currentState
    minChangeInQ = 0
    didConverge = False
    avgReward = totalReward/len(steps)
    for i in range(len(steps)):
        updated = qTable_1[steps[i][0]][steps[i][1]] + alpha*(avgReward-qTable_1[steps[i][0]][steps[i][1]])
        changeInQ = round(abs(qTable_1[currentState][direction] - updated),5)
        if i == 0:
            minChangeInQ = changeInQ
        elif changeInQ < minChangeInQ and changeInQ != 0:
            minChangeInQ = changeInQ
        if changeInQ < convergenceThresh:
            if changeInQ > 0 and pathFound():
                didConverge = True
        qTable_1[steps[i][0]][steps[i][1]] = updated
    timesVisited[currentState] += 1
    return didConverge, minChangeInQ

In [9]:
epsilonValue = 0.3
alpha = 0.6
gamma = 0.7
convergenceThresh = 0.001

In [10]:
qTable_1 = {}
currentState = 0

maxEpisodes = 1000
currentEpisode = 1
converged = False
revistPenalty = -0.25

timesVisited = {}
def resetVisited():
    global timesVisited
    timesVisited = {}
    for i in range(mazeSize[0]*mazeSize[1]):
        timesVisited[i] = 0
resetVisited()

resetTable()
env.reset()
start_time = time.time()
while currentEpisode <= maxEpisodes:
    global currentSteps
    global currentState
    if converged:
        break
        
    direction = nextStep(currentState)
    nextState, reward, terminated, truncated, info = env.step(direction)
    
    if terminated:
        if reward < 1:
            reward = -1
        else:
            reward = 10
            
    if not terminated:
        reward = calcReward(currentState, nextState)
        if timesVisited[nextState] > 0:
            reward += revistPenalty*timesVisited[nextState]
    
    converged, changeInQ = updateTable_qLearning(direction, nextState, reward)
    

    if terminated or truncated or converged:
        observation, info = env.reset()
        currentState = 0
        if not converged:
            currentEpisode += 1
            resetVisited()

        
    if converged:
        end_time = time.time()

    clear_output(wait=True)
    print("Episode: " + str(currentEpisode) + "/" + str(maxEpisodes))
    print("Time: " + str(round(time.time()-start_time, 3)) + " sec")
    print("Q-Table:")
    for i in range(len(qTable_1)):
        print(str(i) + ": " + str(qTable_1[i]))
    print("change in Q: " + str(changeInQ))
            
if converged:
    duration = end_time - start_time
    print(str(round(duration, 3)) + " seconds to converge")
else:
    print("No convergence")

Episode: 32/1000
Time: 52.211 sec
Q-Table:
0: [0, 3.590181270225342, 0.463982178638149, 0]
1: [1.118746474985737, -0.6, -0.046591274481165096, 0]
2: [-0.19284075771494588, -0.6, 0, 0]
3: [0, 0, 0, 0]
4: [0, 3.915104896067066, -0.98976, 0.3614324915871553]
5: [0, 0, 0, 0]
6: [0, 0, 0, 0]
7: [0, 0, 0, 0]
8: [0, -0.98976, 4.585249772501535, 0.8555544917661678]
9: [1.4744926288750682, 5.826446951917581, -0.05518373939797516, -0.9743999999999999]
10: [-0.4875655361181025, -0.6, -0.3482971010998234, -0.84]
11: [-0.3678483438300731, 0, 0, -0.6]
12: [0, 0, 0, 0]
13: [-0.6, 7.409058166597079, 0, 2.5259903202059166]
14: [0, 0, 0, 0]
15: [0, 0, 0, 0]
16: [0, 0, 4.6431187199999995, 0]
17: [-0.6, 0, 9.999580569599999, 0]
18: [0, 0, 0, 0]
19: [0, 0, 0, 0]
change in Q: 0.00063
52.209 seconds to converge


: 